In [22]:
import os
import sys
import copy

import numpy as np
import yt
from yt.frontends.ramses.field_handlers import RTFieldFileHandler

from emission import EmissionLineInterpolator
import galaxy_visualization

In [10]:
filename = "/Users/bnowicki/Documents/Research/Ricotti/output_00273/info_00273.txt"

lines=["H1_6562.80A","O1_1304.86A","O1_6300.30A","O2_3728.80A","O2_3726.10A",
       "O3_1660.81A","O3_1666.15A","O3_4363.21A","O3_4958.91A","O3_5006.84A", 
       "He2_1640.41A","C2_1335.66A","C3_1906.68A","C3_1908.73A","C4_1549.00A",
       "Mg2_2795.53A","Mg2_2802.71A","Ne3_3868.76A","Ne3_3967.47A",
       "N5_1238.82A",
       "N5_1242.80A","N4_1486.50A","N3_1749.67A","S2_6716.44A","S2_6730.82A"]

wavelengths=[6562.80, 1304.86, 6300.30, 3728.80, 3726.10, 1660.81, 1666.15,
             4363.21, 4958.91, 5006.84, 1640.41, 1335.66,
             1906.68, 1908.73, 1549.00, 2795.53, 2802.71, 3868.76,
             3967.47, 1238.82, 1242.80, 1486.50, 1749.67, 6716.44, 6730.82]

cell_fields = [
    "Density",
    "x-velocity",
    "y-velocity",
    "z-velocity",
    "Pressure",
    "Metallicity",
    "xHI",
    "xHII",
    "xHeII",
    "xHeIII",
]

epf = [
    ("particle_family", "b"),
    ("particle_tag", "b"),
    ("particle_birth_epoch", "d"),
    ("particle_metallicity", "d"),
]

# Ionization Parameter Field
# Based on photon densities in bins 2-4
# Don't include bin 1 -> Lyman Werner non-ionizing
def _ion_param(field, data):
    p = RTFieldFileHandler.get_rt_parameters(ds).copy()
    p.update(ds.parameters)

    cgs_c = 2.99792458e10     #light velocity

    # Convert to physical photon number density in cm^-3
    pd_2 = data['ramses-rt','Photon_density_2']*p["unit_pf"]/cgs_c
    pd_3 = data['ramses-rt','Photon_density_3']*p["unit_pf"]/cgs_c
    pd_4 = data['ramses-rt','Photon_density_4']*p["unit_pf"]/cgs_c

    photon = pd_2 + pd_3 + pd_4

    return photon/data['gas', 'number_density']


def _my_temperature(field, data):
    #y(i): abundance per hydrogen atom
    XH_RAMSES=0.76 #defined by RAMSES in cooling_module.f90
    YHE_RAMSES=0.24 #defined by RAMSES in cooling_module.f90
    mH_RAMSES=yt.YTArray(1.6600000e-24,"g") #defined by RAMSES in cooling_module.f90
    kB_RAMSES=yt.YTArray(1.3806200e-16,"erg/K") #defined by RAMSES in cooling_module.f90

    dn=data["ramses","Density"].in_cgs()
    pr=data["ramses","Pressure"].in_cgs()
    yHI=data["ramses","xHI"]
    yHII=data["ramses","xHII"]
    yHe = YHE_RAMSES*0.25/XH_RAMSES
    yHeII=data["ramses","xHeII"]*yHe
    yHeIII=data["ramses","xHeIII"]*yHe
    yH2=1.-yHI-yHII
    yel=yHII+yHeII+2*yHeIII
    mu=(yHI+yHII+2.*yH2 + 4.*yHe) / (yHI+yHII+yH2 + yHe + yel)
    return pr/dn * mu * mH_RAMSES / kB_RAMSES


# TODO see if it works in emission.py
# Luminosity field
# Cloudy Intensity obtained assuming height = 1cm
# Return intensity values erg/s/cm**2
# Multiply intensity at each pixel by volume of pixel -> luminosity
def get_luminosity(line):
   def _luminosity(field, data):
      return data['gas', 'flux_' + line]*data['gas', 'volume']
   return copy.deepcopy(_luminosity)


#number density of hydrogen atoms
def _my_H_nuclei_density(field, data):
    dn=data["ramses","Density"].in_cgs()
    XH_RAMSES=0.76 #defined by RAMSES in cooling_module.f90
    YHE_RAMSES=0.24 #defined by RAMSES in cooling_module.f90
    mH_RAMSES=yt.YTArray(1.6600000e-24,"g") #defined by RAMSES in cooling_module.f90

    return dn*XH_RAMSES/mH_RAMSES


def _pressure(field, data):
    if 'hydro_thermal_pressure' in dir(ds.fields.ramses) and \
        'Pressure' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_thermal_pressure']


def _xHI(field, data):
    if 'hydro_xHI' in dir(ds.fields.ramses) and \
        'xHI' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHI']


def _xHII(field, data):
    if 'hydro_xHII' in dir(ds.fields.ramses) and \
        'xHII' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHII']


def _xHeII(field, data):
    if 'hydro_xHeII' in dir(ds.fields.ramses) and \
        'xHeII' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHeII']


def _xHeIII(field, data):
    if 'hydro_xHeIII' in dir(ds.fields.ramses) and \
        'xHeIII' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHeIII']


In [12]:
'''
-------------------------------------------------------------------------------
Load Simulation Data
Add Derived Fields
-------------------------------------------------------------------------------
'''

ds = yt.load(filename, extra_particle_fields=epf)

# Ionization parameter
ds.add_field(
    ('gas', 'ion_param'),
    function=_ion_param,
    sampling_type="cell",
    units="cm**3",
    force_override=True
)

ds.add_field(
    ("gas","my_temperature"),
    function=_my_temperature,
    sampling_type="cell",
    # TODO units
    #units="K",
    #units="K*cm**3/erg",
    units='K*cm*dyn/erg',
    force_override=True
)

ds.add_field(
    ("gas","my_H_nuclei_density"),
    function=_my_H_nuclei_density,
    sampling_type="cell",
    units="1/cm**3",
    force_override=True
)

ds.add_field(
    ("gas","number_density"),
    function=_my_H_nuclei_density,
    sampling_type="cell",
    units="1/cm**3",
    force_override=True
)


ds.add_field(
    ("ramses","Pressure"),
    function=_pressure,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHI"),
    function=_xHI,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHII"),
    function=_xHII,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHeII"),
    function=_xHeII,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHeIII"),
    function=_xHeIII,
    sampling_type="cell",
    units="1",
    #force_override=True
)


# Normalize by Density Squared Flag
dens_normalized = True
if dens_normalized: 
    units = '1/cm**6'
else:
    units = '1'

# Instance of EmissionLineInterpolator for line list at filename
line_list = os.path.join(os.getcwd(), 'linelist.dat')
emission_interpolator = EmissionLineInterpolator(line_list, lines)

# Add flux and luminosity fields for all lines in the list
for i, line in enumerate(lines):
    ds.add_field(
        ('gas', 'flux_' + line),
        function=emission_interpolator.get_line_emission(
            i, dens_normalized=dens_normalized
        ),
        sampling_type='cell',
        units=units,
        force_override=True
    )
    # TODO change get_line_emission to accept line not idx

    ds.add_field(
        ('gas', 'luminosity_' + line),
        function=emission_interpolator.get_luminosity(lines[i]),
        #function=get_luminosity(lines[i]),
        sampling_type='cell',
        units='1/cm**3',
        force_override=True
    )

ad = ds.all_data()
print(ds.field_list)


yt : [INFO     ] 2025-03-26 16:36:34,190 Parameters: current_time              = 0.3604448649237178 Gyr
yt : [INFO     ] 2025-03-26 16:36:34,190 Parameters: domain_dimensions         = [64 64 64]
yt : [INFO     ] 2025-03-26 16:36:34,190 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-03-26 16:36:34,191 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-03-26 16:36:34,191 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-03-26 16:36:34,192 Parameters: current_redshift          = 12.171087046255657
yt : [INFO     ] 2025-03-26 16:36:34,192 Parameters: omega_lambda              = 0.685000002384186
yt : [INFO     ] 2025-03-26 16:36:34,192 Parameters: omega_matter              = 0.314999997615814
yt : [INFO     ] 2025-03-26 16:36:34,193 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-03-26 16:36:34,193 Parameters: hubble_constant           = 0.674000015258789
yt : [INFO     ] 2025-03-26 16:36:34,203 Detected RA

[]


yt : [WARNING  ] 2025-03-26 16:36:34,487 The DM particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,487 The star particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,487 The cloud particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,487 The dust particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,488 The star_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,488 The cloud_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,488 The dust_tracer particle filter already exists. Overriding.
yt : [WARNING  ] 2025-03-26 16:36:34,489 The gas_tracer particle filter already exists. Overriding.
yt : [INFO     ] 2025-03-26 16:36:34,489 Adding particle_type: DM
yt : [INFO     ] 2025-03-26 16:36:34,498 Adding particle_type: star
yt : [INFO     ] 2025-03-26 16:36:34,504 Adding particle_type: cloud
yt : [INFO     ] 2

minU=-6.0, maxU=1.0, stepU=0.5, minN=-1.0, maxN=6.0, stepN=0.5, minT=3.0, maxT=6.0, stepT=0.1
Line List Shape = (25, 6975)
15 15 31
[('gravity', 'Potential'), ('gravity', 'x-acceleration'), ('gravity', 'y-acceleration'), ('gravity', 'z-acceleration'), ('io', 'particle_birth_epoch'), ('io', 'particle_family'), ('io', 'particle_identity'), ('io', 'particle_mass'), ('io', 'particle_metallicity'), ('io', 'particle_position_x'), ('io', 'particle_position_y'), ('io', 'particle_position_z'), ('io', 'particle_refinement_level'), ('io', 'particle_tag'), ('io', 'particle_velocity_x'), ('io', 'particle_velocity_y'), ('io', 'particle_velocity_z'), ('nbody', 'particle_mass'), ('nbody', 'particle_position_x'), ('nbody', 'particle_position_y'), ('nbody', 'particle_position_z'), ('nbody', 'particle_velocity_x'), ('nbody', 'particle_velocity_y'), ('nbody', 'particle_velocity_z'), ('ramses', 'Density'), ('ramses', 'Metallicity'), ('ramses', 'Pressure'), ('ramses', 'refinement-param'), ('ramses', 'x-velo

In [ ]:
'''
-------------------------------------------------------------------------------
Run routines on data
-------------------------------------------------------------------------------
'''

viz = galaxy_visualization.VisualizationManager(filename, lines, wavelengths)
star_ctr = viz.star_center(ad)
sp = ds.sphere(star_ctr, (3000, "pc"))
sp_lum = ds.sphere(star_ctr, (10, 'kpc'))
width = (1500, 'pc')

sim_run = filename.split('/')[-1]

field_list = [
    ('gas', 'temperature'),
    ('gas', 'density'),
    ('gas', 'my_H_nuclei_density'),
    ('gas', 'my_temperature'),
    ('gas', 'ion_param'),
    ('gas', 'metallicity')
]

weight_field_list = [
    ('gas', 'my_H_nuclei_density'),
    ('gas', 'my_H_nuclei_density'),
    ('gas', 'my_H_nuclei_density'),
    ('gas', 'my_H_nuclei_density'),
    ('gas', 'my_H_nuclei_density'),
    ('gas', 'my_H_nuclei_density')
]

title_list = [
    'Temperature [K]',
    r'Density [g cm$^{-3}$]',
    r'H Nuclei Number Density [cm$^{-3}$]',
    'My Temperature [K]',
    'Ionization Parameter',
    'Metallicity'
]

for line in lines:
    if line == 'H1_6562.80A':
        line_title = r'H$\alpha$_6562.80A'
    else:
        line_title = line

    field_list.append(('gas', 'flux_'  + line))
    title_list.append(line_title.replace('_', ' ') + 
                      r' Flux [$erg\: s^{-1}\: cm^{-2}$]')
    weight_field_list.append(None)

    field_list.append(('gas', 'luminosity_'  + line))
    title_list.append(line_title.replace('_', ' ') + 
                      r' Luminosity [$erg\: s^{-1}$]')
    weight_field_list.append(None)


viz.save_sim_info(ds)
viz.plot_wrapper(ds, sp, width, star_ctr, field_list,
                     weight_field_list, title_list, proj=True, slc=False)


yt : [INFO     ] 2025-03-26 16:37:34,075 Identified   384/  384 intersecting domains (  385 through hilbert key indexing)


Filename = /Users/bnowicki/Documents/Research/Ricotti/output_00273/info_00273.txt
File Directory = /Users/bnowicki/Documents/Research/Ricotti/output_00273
Output File = output_00273
Simulation Run = 00273
Analysis Directory = analysis/output_00273_analysis


yt : [INFO     ] 2025-03-26 16:37:34,782 Identified   375/  384 intersecting domains (  375 through hilbert key indexing)
/Users/bnowicki/miniforge3/lib/python3.10/site-packages/yt/frontends/ramses/fields.py:241: RuntimeWarning: Trying to calculate temperature but the cooling tables couldn't be found or read. yt will return T/µ instead of T — this is equivalent to assuming µ=1.0. To suppress this, derive the temperature from temperature_over_mu with some values for mu.
  warnings.warn(
yt : [INFO     ] 2025-03-26 16:37:37,434 Projection completed
yt : [INFO     ] 2025-03-26 16:37:37,440 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:37:37,440 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:37:37,442 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:37:37,442 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:37:37,445 Making a fixed resolution buffer of (('gas', 'temperature')) 1000 by 1000
yt : [INFO     ] 2025-03-26 16:37:38,820 Projection completed
yt : [

yt : [INFO     ] 2025-03-26 16:39:04,816 Projection completed
yt : [INFO     ] 2025-03-26 16:39:04,817 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:04,817 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:04,818 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:04,819 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:04,820 Making a fixed resolution buffer of (('gas', 'luminosity_C4_1549.00A')) 1000 by 1000


yt : [INFO     ] 2025-03-26 16:39:07,460 Projection completed
yt : [INFO     ] 2025-03-26 16:39:07,461 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:07,461 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:07,463 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:07,463 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:07,464 Making a fixed resolution buffer of (('gas', 'flux_Mg2_2795.53A')) 1000 by 1000
yt : [INFO     ] 2025-03-26 16:39:10,278 Projection completed
yt : [INFO     ] 2025-03-26 16:39:10,278 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:10,278 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:10,280 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:10,280 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:10,281 Making a fixed resolution buffer of (('gas', 'luminosity_Mg2_2795.53A')) 1000 by 1000
yt : [INFO     ] 2025-03-26 16:39:12,729 Projection completed
yt : [INFO     ] 2025-03-26 16:39:12,7

yt : [INFO     ] 2025-03-26 16:39:31,923 Projection completed
yt : [INFO     ] 2025-03-26 16:39:31,924 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:31,924 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:31,926 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:31,926 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:31,927 Making a fixed resolution buffer of (('gas', 'luminosity_N5_1238.82A')) 1000 by 1000


yt : [INFO     ] 2025-03-26 16:39:34,507 Projection completed
yt : [INFO     ] 2025-03-26 16:39:34,507 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:34,507 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:34,508 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:34,509 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:34,509 Making a fixed resolution buffer of (('gas', 'flux_N5_1242.80A')) 1000 by 1000


yt : [INFO     ] 2025-03-26 16:39:37,767 Projection completed
yt : [INFO     ] 2025-03-26 16:39:37,767 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:37,768 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:37,769 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:37,769 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:37,770 Making a fixed resolution buffer of (('gas', 'luminosity_N5_1242.80A')) 1000 by 1000


yt : [INFO     ] 2025-03-26 16:39:40,434 Projection completed
yt : [INFO     ] 2025-03-26 16:39:40,435 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:40,435 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:40,436 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:40,437 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:40,437 Making a fixed resolution buffer of (('gas', 'flux_N4_1486.50A')) 1000 by 1000
yt : [INFO     ] 2025-03-26 16:39:43,233 Projection completed
yt : [INFO     ] 2025-03-26 16:39:43,233 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:43,233 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:43,235 xlim = 0.490978 0.491364
yt : [INFO     ] 2025-03-26 16:39:43,235 ylim = 0.492583 0.492970
yt : [INFO     ] 2025-03-26 16:39:43,236 Making a fixed resolution buffer of (('gas', 'luminosity_N4_1486.50A')) 1000 by 1000
yt : [INFO     ] 2025-03-26 16:39:45,809 Projection completed
yt : [INFO     ] 2025-03-26 16:39:45,809

In [17]:
extrema = {('gas', 'my_temperature'): (1e3, 1e8),
           ('gas', 'my_H_nuclei_density'): (1e-4, 1e4)}

line_title = r'H$\alpha$_6562.80A'

viz.phase_plot(ds, sp, x_field=('gas', 'my_temperature'),
               y_field=('gas', 'my_H_nuclei_density'), z_field=('gas', 'flux_H1_6562.80A'),
               extrema=extrema, x_label='Temperature [K]', 
               y_label=r'H Nuclei Number Density [cm$^{-3}$]', 
               z_label=line_title.replace('_', ' ') + 
                      r' Flux [erg s$^{-1}$ cm$^{-2}$]')

yt : [INFO     ] 2025-03-26 16:52:52,237 Saving plot analysis/output_00273_analysis/output_00273_my_temperature_my_H_nuclei_density_flux_H1_6562.80A_phase.png


In [18]:
viz.calc_luminosities(sp)

H1_6562.80A Luminosity = 8.834081984412808e+41 cm**(-3) erg/s
O1_1304.86A Luminosity = 5.130504462828963e+34 cm**(-3) erg/s
O1_6300.30A Luminosity = 8.398072130951248e+36 cm**(-3) erg/s
O2_3728.80A Luminosity = 1.4453069758064272e+37 cm**(-3) erg/s
O2_3726.10A Luminosity = 2.6456051389974314e+37 cm**(-3) erg/s
O3_1660.81A Luminosity = 7.609348442388433e+35 cm**(-3) erg/s
O3_1666.15A Luminosity = 2.2284170286228816e+36 cm**(-3) erg/s
O3_4363.21A Luminosity = 8.756333174869612e+35 cm**(-3) erg/s
O3_4958.91A Luminosity = 8.348039861471982e+36 cm**(-3) erg/s
O3_5006.84A Luminosity = 2.4907174062091733e+37 cm**(-3) erg/s
He2_1640.41A Luminosity = 1.4251693123545895e+38 cm**(-3) erg/s
C2_1335.66A Luminosity = 4.187353508012613e+35 cm**(-3) erg/s
C3_1906.68A Luminosity = 1.3738911794389046e+37 cm**(-3) erg/s
C3_1908.73A Luminosity = 1.1674005577107624e+37 cm**(-3) erg/s
C4_1549.00A Luminosity = 3.985881378576702e+29 cm**(-3) erg/s
Mg2_2795.53A Luminosity = 3.5391771044188585e+37 cm**(-3) erg/

[array(8.83408198e+41),
 array(5.13050446e+34),
 array(8.39807213e+36),
 array(1.44530698e+37),
 array(2.64560514e+37),
 array(7.60934844e+35),
 array(2.22841703e+36),
 array(8.75633317e+35),
 array(8.34803986e+36),
 array(2.49071741e+37),
 array(1.42516931e+38),
 array(4.18735351e+35),
 array(1.37389118e+37),
 array(1.16740056e+37),
 array(3.98588138e+29),
 array(3.5391771e+37),
 array(1.78793082e+37),
 array(5.25355092e+35),
 array(1.59453244e+35),
 array(9.19283542e+32),
 array(4.58610157e+32),
 array(9.81514974e+32),
 array(1.18496299e+36),
 array(8.41941697e+36),
 array(1.28071921e+37)]

In [23]:
viz.save_sim_field_info(ds, sp)

IndexError: tuple index out of range